<a href="https://colab.research.google.com/github/amrahmani/Pythorch/blob/main/Ch5_RNN_WeatherPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem: a one-day-ahead temperature prediction problem**


Using PyTorch, create an **RNN** and write code to train the model based on this data (https://raw.githubusercontent.com/amrahmani/Pythorch/main/WeatherHistory.csv), and test it for the problem. The model has four inputs: temperature (°C), humidity, wind speed (km/h), and pressure (millibars), respectively, and one output: "one-hour ahead temperature (°C)".

**Import Libraries:**

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

**Load and Prepare Dataset**

In [ ]:
# Load dataset
url = 'https://raw.githubusercontent.com/amrahmani/Pythorch/main/WeatherHistory.csv'

# Read the CSV file into a Pandas DataFrame (assuming no header row)
data = pd.read_csv(url)

# Assuming your dataset columns are: 'Temperature', 'Humidity', 'Wind Speed', 'Pressure', 'Six-hour ahead Temperature'
# Separate inputs and outputs
X = data[["Temperature (C)", "Humidity", "Wind Speed (km/h)", "Pressure (millibars)"]].values
y = data[['one-hour ahead Temperature (C)']].values

# Normalization
# Scale the input features and target variable to the range [0, 1]; X_norm = (X - X_min) / (X_max - X_min)
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# Convert the numpy arrays to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# Use the last 21 to 1 rows as test data
X_test = X[-21:-1]
y_test = y[-21:-1]

# Use the remaining data as training data
X_train = X[:-21]
y_train = y[:-21]

# Create a DataLoader for the training data to facilitate mini-batch training
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=32, shuffle=False) # Shuffling helps reduce the impact of data order on model training and can potentially improve generalization.


**Define and Train the RNN**

In [ ]:
class WeatherRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(WeatherRNN, self).__init__()
        self.num_layers = num_layers
        # Define the dimension of the hidden state for each layer
        self.hidden_size = hidden_size
               # to recap for MLP we have self.hidden = nn.Linear(2, 5) it defines the hidden layer with 5 neurons and input size 2
               # to recap for MLP we have self.output = nn.Linear(5, 1) it defines the output layer with 1 neuron
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size) # fc = Fully connected

    # By default, the nn.RNN module in PyTorch uses the hyperbolic tangent (tanh) activation function
    def forward(self, x):
        # Initialize hidden state with zeros, h0 represents the initial hidden state of an RNN
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        # RNN forward pass
        out, _ = self.rnn(x, h0) # It returns two tensors, and assigns the first tensor to out and discards the second one (might contain the final hidden state) using the _
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :]) #  It selects all elements from the last row (-1)
        return out

# Hyperparameters
input_size = 4      # Number of input features
hidden_size = 24    # Number of features in the hidden state
output_size = 1     # Single output (next hour temperature)
num_layers = 1      # Number of layers in the RNN
num_epochs = 100    # Number of epochs to train
learning_rate = 0.001  # Learning rate

model = WeatherRNN(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

# Training loop
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        # Reshape input to (number of elements in the 1st dimention, sequence_length, input_size)
        X_batch = X_batch.view(X_batch.size(0), 1, input_size)

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass and optimization
        # Zero the gradients to prevent accumulation
        optimizer.zero_grad()
        # Compute gradients of the loss w.r.t. model parameters
        loss.backward()
        # Update model parameters using optimizer
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.0230
Epoch [20/100], Loss: 0.0115
Epoch [30/100], Loss: 0.0066
Epoch [40/100], Loss: 0.0045
Epoch [50/100], Loss: 0.0033
Epoch [60/100], Loss: 0.0028
Epoch [70/100], Loss: 0.0026
Epoch [80/100], Loss: 0.0025
Epoch [90/100], Loss: 0.0025
Epoch [100/100], Loss: 0.0025


**Test the RNN and Compare Predictions**

In [ ]:

# When we don't need to compute gradients during the forward pass
with torch.no_grad():
    X_test = X_test.view(X_test.size(0), 1, input_size)
    predictions = model(X_test)
    actuals = y_test

    # Rescale predictions and actuals back to original scale (Denormalization)
    predictions = scaler_y.inverse_transform(predictions)
    actuals = scaler_y.inverse_transform(actuals)

    # Compare predictions and actual values
    print('Predictions vs   Actuals         Comparison')
    for i in range(len(predictions)):
      print(predictions[i], '\t', actuals[i], '\t', predictions[i]-actuals[i])


Predictions vs   Actuals         Comparison
[9.88473949] 	 [9.46000015] 	 [0.42473934]
[9.23584244] 	 [10.68000004] 	 [-1.4441576]
[10.27471362] 	 [8.93999999] 	 [1.33471364]
[8.27363962] 	 [8.93999999] 	 [-0.66636037]
[8.66692865] 	 [12.49999986] 	 [-3.83307121]
[11.93449353] 	 [16.03000026] 	 [-4.09550674]
[15.27250476] 	 [19.86000005] 	 [-4.58749529]
[18.80380349] 	 [21.87999945] 	 [-3.07619596]
[20.91589239] 	 [22.80000035] 	 [-1.88410796]
[21.69212646] 	 [23.96000013] 	 [-2.26787367]
[22.738135] 	 [24.92000032] 	 [-2.18186532]
[23.67067069] 	 [24.96999983] 	 [-1.29932914]
[23.73514203] 	 [25.03999976] 	 [-1.30485774]
[23.85297008] 	 [24.82999996] 	 [-0.97702989]
[23.90418488] 	 [22.22000046] 	 [1.68418442]
[21.58680838] 	 [18.86999999] 	 [2.71680838]
[18.33354984] 	 [17.13999976] 	 [1.19355008]
[16.77170698] 	 [14.9300002] 	 [1.84170678]
[14.46593402] 	 [15.41999974] 	 [-0.95406572]
[15.22364837] 	 [14.17999981] 	 [1.04364856]
